<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import numpy as np

def get_split2(data, feature, split_val):

  right = data[data[:,feature] >= split_val]

  left = data[data[:,feature] < split_val]

  return left, right


def calc_gini(left, right, classes):

  tot_counts = float(len(left) + len(right))



  gini = 0.0

  if left.shape[0] == 0 or right.shape[0] == 0: 
    
    return 1e4


  left_0 = np.count_nonzero(left[:,-1] == class_values[0])/left.shape[0]
  left_1 = np.count_nonzero(left[:,-1] == class_values[1])/left.shape[0]

  right_0 = np.count_nonzero(right[:,-1] == class_values[0])/right.shape[0]
  right_1 = np.count_nonzero(right[:,-1] == class_values[1])/right.shape[0]

  gini = (1 - (left_0**2+left_1**2))*(left.shape[0]/tot_counts) + (1 - (right_0**2+right_1**2))*(right.shape[0]/tot_counts)

  return gini



def get_split(data):

  

  classes = np.unique(data[:,-1])

  best_idx, best_val, best_gini_score, best_groups = 1e3,1e3,1e3,0

  for feature in range(data.shape[1]-1):
    for row in data:
      left, right = get_split2(data, feature, row[feature])
      gini = calc_gini(left, right, classes)
      print('X%d < %.3f Gini=%.3f' % ((feature+1), row[feature], gini),  feature)

      if gini < best_gini_score:
        best_idx, best_val, best_gini_score, best_groups = feature, row[feature], gini, [left, right]
  return {'index':best_idx, 'value':best_val, 'groups':best_groups, 'gini':best_gini_score}
 
dataset = np.array([[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]])
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))




class node():

  def __init__(self, left=0, right=0, feature, val, depth, data):

    if left == 0 or right == 0:
      self.is_leaf = 1

    self. left = left
    self.right = right

X1 < 2.771 Gini=0.444 0
X1 < 1.729 Gini=10000.000 0
X1 < 3.678 Gini=0.286 0
X1 < 3.961 Gini=0.167 0
X1 < 2.999 Gini=0.375 0
X1 < 7.498 Gini=0.286 0
X1 < 9.002 Gini=0.375 0
X1 < 7.445 Gini=0.167 0
X1 < 10.125 Gini=0.444 0
X1 < 6.642 Gini=0.000 0
X2 < 1.785 Gini=0.500 1
X2 < 1.170 Gini=0.444 1
X2 < 2.813 Gini=0.320 1
X2 < 2.620 Gini=0.417 1
X2 < 2.209 Gini=0.476 1
X2 < 3.163 Gini=0.167 1
X2 < 3.339 Gini=0.444 1
X2 < 0.477 Gini=10000.000 1
X2 < 3.235 Gini=0.286 1
X2 < 3.320 Gini=0.375 1
Split: [X1 < 6.642]


In [132]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right
 
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
  # count all samples at split point
  n_instances = float(sum([len(group) for group in groups]))
  # sum weighted Gini index for each group
  gini = 0.0
  for group in groups:
    size = float(len(group))
    # avoid divide by zero
    if size == 0:
      continue
    score = 0.0
    # score the group based on the score for each class
    for class_val in classes:
      p = [row[-1] for row in group].count(class_val) / size
      score += p * p
    # weight the group score by its relative size
    gini += (1.0 - score) * (size / n_instances)
  return gini
 
# Select the best split point for a dataset
def get_split(dataset):
  class_values = list(set(row[-1] for row in dataset))
  b_index, b_value, b_score, b_groups = 999, 999, 999, None
  for index in range(len(dataset[0])-1):
    for row in dataset:
      groups = test_split(index, row[index], dataset)
      #groups2 = get_split2(np.array(dataset), index, row[index])
      gini = gini_index(groups, class_values)
      gini2 = calc_gini(np.array(groups[0]), np.array(groups[1]), class_values)
      print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
      if gini < b_score:
        b_index, b_value, b_score, b_groups = index, row[index], gini, groups
  return {'index':b_index, 'value':b_value, 'groups':b_groups}

dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

X1 < 2.771 Gini=0.444
X1 < 1.729 Gini=0.500
X1 < 3.678 Gini=0.286
X1 < 3.961 Gini=0.167
X1 < 2.999 Gini=0.375
X1 < 7.498 Gini=0.286
X1 < 9.002 Gini=0.375
X1 < 7.445 Gini=0.167
X1 < 10.125 Gini=0.444
X1 < 6.642 Gini=0.000
X2 < 1.785 Gini=0.500
X2 < 1.170 Gini=0.444
X2 < 2.813 Gini=0.320
X2 < 2.620 Gini=0.417
X2 < 2.209 Gini=0.476
X2 < 3.163 Gini=0.167
X2 < 3.339 Gini=0.444
X2 < 0.477 Gini=0.500
X2 < 3.235 Gini=0.286
X2 < 3.320 Gini=0.375
Split: [X1 < 6.642]


In [123]:
class_values = list(set(row[-1] for row in dataset))
b_index, b_value, b_score, b_groups = 999, 999, 999, None
index = list(range(len(dataset[0])-1))[0]
row = dataset[0]
groups = test_split(index, row[index], dataset)
#groups2 = get_split2(np.array(dataset), index, row[index])
# count all samples at split point
n_instances = float(sum([len(group) for group in groups]))
print('n_inst: ', n_instances)
# sum weighted Gini index for each group
gini = 0.0
for group in groups:
  size = float(len(group))
# avoid divide by zero
  if size == 0:
    continue
  score = 0.0
# score the group based on the score for each class
  for class_val in class_values:
    p = [row[-1] for row in group].count(class_val) / size
    print('p: ', p)
    score += p * p
# weight the group score by its relative size
  
  gini += (1.0 - score) * (size / n_instances)
  print('div: ', size / n_instances)
  print('score:', score, 'gini: ', gini)


print('gini: ', gini )

print('--------------------------------------------')

left, right = get_split2(np.array(dataset), index, row[index])

tot_counts = float(len(left) + len(right))

print('tot_counts: ', tot_counts)
print('left_shape', left.shape[0])

gini = 0.0


left_0 = np.count_nonzero(left[:,-1] == class_values[0])/left.shape[0]
left_1 = np.count_nonzero(left[:,-1] == class_values[1])/left.shape[0]

right_0 = np.count_nonzero(right[:,-1] == class_values[0])/right.shape[0]
right_1 = np.count_nonzero(right[:,-1] == class_values[1])/right.shape[0]

gini = (1 - (left_0**2+left_1**2))*(left.shape[0]/tot_counts) + (1 - (right_0**2+right_1**2))*(right.shape[0]/tot_counts)

print('p_ele: ' ,1-(right_0**2+right_1**2),(1.0 - right_0**2+right_1**2), (1.0 - (right_0**2+right_1**2))*(right.shape[0]/tot_counts))

print('gini1: ', gini)


n_inst:  10.0
p:  1.0
p:  0.0
div:  0.1
score: 1.0 gini:  0.0
p:  0.4444444444444444
p:  0.5555555555555556
div:  0.9
score: 0.5061728395061729 gini:  0.4444444444444444
gini:  0.4444444444444444
--------------------------------------------
tot_counts:  10.0
left_shape 1
p_ele:  0.49382716049382713 1.1111111111111112 0.4444444444444444
gini1:  0.4444444444444444
